In [48]:
LISTINGS_METADATA_PATH = "abo-listings/listings/metadata"
import json
import os

listings_metadata = []
for file in os.listdir(LISTINGS_METADATA_PATH):
  if file.endswith(".json"):
    with open(os.path.join(LISTINGS_METADATA_PATH, file), "r") as f:
      # filter out listings with furniture in them
      for line in f:
        try:
          temp = json.loads(line)
          if "furniture" in str(temp).lower():
            listings_metadata.append(temp)
        except Exception as e:
          print(file, e)

print(len(listings_metadata))

Untitled-1.json Expecting property name enclosed in double quotes: line 2 column 1 (char 2)
Untitled-1.json Extra data: line 1 column 20 (char 19)
Untitled-1.json Extra data: line 1 column 13 (char 12)
Untitled-1.json Extra data: line 1 column 25 (char 24)
Untitled-1.json Extra data: line 1 column 15 (char 14)
Untitled-1.json Extra data: line 1 column 16 (char 15)
Untitled-1.json Expecting value: line 1 column 7 (char 6)
Untitled-1.json Extra data: line 1 column 13 (char 12)
Untitled-1.json Extra data: line 1 column 14 (char 13)
Untitled-1.json Expecting value: line 1 column 5 (char 4)
Untitled-1.json Extra data: line 1 column 13 (char 12)
Untitled-1.json Extra data: line 1 column 25 (char 24)
Untitled-1.json Extra data: line 1 column 15 (char 14)
Untitled-1.json Extra data: line 1 column 16 (char 15)
Untitled-1.json Expecting value: line 1 column 7 (char 6)
Untitled-1.json Extra data: line 1 column 13 (char 12)
Untitled-1.json Extra data: line 1 column 14 (char 13)
Untitled-1.json Exp

In [49]:
from collections import defaultdict
keys = listings_metadata[0].keys()
key_map = defaultdict(list)
for listing in listings_metadata:
  for key in listing.keys():
    key_map[key].append(listing["item_id"])

for key, value in sorted(key_map.items(), key=lambda x: len(x[1]), reverse=True):
  print(key, len(value))


item_id 9242
item_name 9242
product_type 9242
country 9242
marketplace 9242
domain_name 9242
brand 9233
main_image_id 9161
node 8807
model_number 8456
other_image_id 8367
item_keywords 8011
bullet_point 7635
color 7500
item_dimensions 7212
item_weight 6882
material 5501
3dmodel_id 4494
spin_id 4473
style 4098
color_code 3361
item_shape 1533
fabric_type 1429
model_name 649
finish_type 405
pattern 391
product_description 265
model_year 12


In [50]:
def get_unique_words(texts):
  unique_words = []
  for text in texts:
    if text["language_tag"].startswith("en"):
      words = text["value"].replace("\n", " ").replace(",", " ").replace(".", " ").split(" ")
      for word in words:
        if word not in unique_words:
          unique_words.append(word.lower())
  return " ".join(unique_words)


In [51]:
filtered_metadata = []
dimensions_units_multiplier = {"inches": 1, "feet": 12, "centimeters": 0.393701, "millimeters": 0.0393701}
for listing in listings_metadata:
  # item_dimensions, material, color, fabric_type, finish_type, pattern, style, item_shape, item_keywords, bullet_points
  # color_code
  temp = {}
  temp["item_id"] = listing["item_id"]
  if "item_dimensions" not in listing:
    continue
  dimensions = {}
  if "height" in listing["item_dimensions"]:
      dimensions["height"] = listing["item_dimensions"]["height"]["value"] * dimensions_units_multiplier[listing["item_dimensions"]["height"]["unit"]]
  if "width" in listing["item_dimensions"]:
    dimensions["width"] = listing["item_dimensions"]["width"]["value"] * dimensions_units_multiplier[listing["item_dimensions"]["width"]["unit"]]
  if "length" in listing["item_dimensions"]:
    dimensions["length"] = listing["item_dimensions"]["length"]["value"] * dimensions_units_multiplier[listing["item_dimensions"]["length"]["unit"]]
  temp["dimensions"] = dimensions
  
  description = ""
  if "material" in listing:
    material = get_unique_words(listing["material"])
    description += material + " "
    temp["material"] = material
  if "color" in listing:
    color =   get_unique_words(listing["color"])
    description += color + " "
    temp["color"] = color
  if "fabric_type" in listing:
    fabric_type = get_unique_words(listing["fabric_type"])
    description += fabric_type + " "
    temp["fabric_type"] = fabric_type
  if "finish_type" in listing:
    finish_type = get_unique_words(listing["finish_type"])
    description += finish_type + " "
    temp["finish_type"] = finish_type
  if "pattern" in listing:
    pattern = get_unique_words(listing["pattern"])
    description += pattern + " "
    temp["pattern"] = pattern
  if "style" in listing:
    style = get_unique_words(listing["style"])
    description += style + " "
    temp["style"] = style
  if "item_shape" in listing:
    item_shape = get_unique_words(listing["item_shape"])
    description += item_shape + " "
    temp["item_shape"] = item_shape
  if "item_keywords" in listing:
    item_keywords = get_unique_words(listing["item_keywords"])
    description += item_keywords + " "
    temp["item_keywords"] = item_keywords
  if "bullet_points" in listing:
    bullet_points = get_unique_words(listing["bullet_points"])
    description += bullet_points + " "
    temp["bullet_points"] = bullet_points
    
  temp["description"] = description
  filtered_metadata.append(temp)
print(len(filtered_metadata))

7212


In [55]:
# with open("filtered_metadata.json", "w") as f:
#   json.dump(filtered_metadata, f)
filtered_metadata[0]

{'item_id': 'B0853VPXF3',
 'dimensions': {'height': 36.6, 'width': 37.4015748, 'length': 34.2519685},
 'color': 'Hemp',
 'item_keywords': 'studio recliner big egg signature seniors elderly lazy boy baxton modern furniture design by chairs recliners power man massage farmhouse ashley yandel sofa set electric gdf leather 2 kieran oversized lift',
 'description': 'Hemp studio recliner big egg signature seniors elderly lazy boy baxton modern furniture design by chairs recliners power man massage farmhouse ashley yandel sofa set electric gdf leather 2 kieran oversized lift '}

In [46]:
import json

import torch
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

with open("embedded_data_w_3d.json", "r") as f:
    data = json.load(f)
stored_embeddings = torch.tensor([item["embedding"] for item in data])
print(type(stored_embeddings))
def simple_retriever(query: str = "yellow chair with wheels", top_k: int = 50):
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    hits = util.semantic_search(query_embedding, stored_embeddings, top_k=top_k)
    hits = hits[0]
    
    results = []
    for hit in hits:
        idx = hit['corpus_id']  # index of the stored item
        score = hit['score']
        results.append((data[idx], score))
    return results
retrieved = simple_retriever()
# print(retrieved)
[ (i[0]["item_id"], i[0]["description"]) for i in retrieved]

<class 'torch.Tensor'>


[('B07Y3QQXY8', 'Yellow Stack Resin Chair with Chrome Legs '),
 ('B07Y3Q8JWL', 'Yellow Stack Resin Chair with Chrome Legs '),
 ('B07Y3QMGYT', 'Yellow Stack Resin Chair with Chrome Legs '),
 ('B07Y3R8HYC', 'Yellow Stack Resin Chair with Chrome Legs '),
 ('B082VFS7ZW', 'Steel Grey Swivel Chair '),
 ('B07QBMQ75C', 'Light Grey R6478A30 manning recliner living room chair '),
 ('B07P5LM551',
  'Wood Yellow Fabric Mid-Century modern ottomans cushion yellow upholstered stool '),
 ('B07B8445B5',
  ' Yellow 0 Round dining-chairs bar stools set of 2 counter height stool barstools 4 3 Rivet mid century modern Yellow '),
 ('B07MQD29CZ',
  'Yellow and Cream Fabric Round vanity chair step stool folding rolling salon fur with wheels pink office corsair t2 collapsible desk adjustable foldable for kids '),
 ('B085FHV3FH',
  'Wood Black Fabric Modern Rectangular upholstered swivel gliding chair '),
 ('B085FHGH3V',
  'Wood Black Fabric Transitional Rectangular upholstered swivel gliding chair '),
 ('B07Y5

In [53]:
import json
import os
# with open("filtered_metadata.json", "r") as f:
#   filtered_metadata = json.load(f)
ids = [item["item_id"] for item in filtered_metadata]
_3D_PATH = "/Volumes/X9 Pro/isr dataset/3dmodels/original/"
_3d_paths = []
_3d_ids = []
for dir in os.listdir(_3D_PATH):
  if os.path.isdir(os.path.join(_3D_PATH, dir)):
    for file in os.listdir(os.path.join(_3D_PATH, dir)):
      if file.split(".")[0] in ids:
        _3d_ids.append(file.split(".")[0])
        _3d_paths.append(os.path.join(_3D_PATH, dir, file))
print(len(_3d_paths))
print(len(_3d_ids))
# with open("3d_paths.json", "w") as f:
#   json.dump(_3d_paths, f)


4439
4439


In [54]:
_3d_available_metadata = []
for listing in filtered_metadata:
  if listing["item_id"] in _3d_ids:
    _3d_available_metadata.append(listing)
print(len(_3d_available_metadata))


4440


In [56]:
from sentence_transformers import SentenceTransformer

# sentences = [listing["description"] for listing in _3d_available_metadata]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings_data = []

for obj in _3d_available_metadata:
  obj["embedding"] = model.encode(obj.get("description", "")).tolist()  
  embeddings_data.append(obj)
with open("embedded_data.json", 'w', encoding='utf-8') as f:
    json.dump(embeddings_data, f)

In [57]:
import shutil
destination_directory = '/Volumes/X9 Pro/isr dataset/filtered/'

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Loop over the file paths and copy each file
for file_path in _3d_paths:
    if os.path.isfile(file_path):
        shutil.copy(file_path, destination_directory)
        # print(f"Copied {file_path} to {destination_directory}")
    else:
        print(f"File not found: {file_path}")

In [8]:
!python3.12 designer.py

```json
[
    {
        "name": "Bed",
        "description": "A queen-sized bed for comfortable sleeping."
    },
    {
        "name": "Wardrobe",
        "description": "A tall wardrobe for clothing storage."
    },
    {
        "name": "Desk",
        "description": "A minimalist desk for work or study."
    },
    {
        "name": "Chair",
        "description": "An ergonomic chair for the desk."
    },
    {
        "name": "Nightstand",
        "description": "A small nightstand for bedside essentials."
    }
]
```
[{'name': 'Bed', 'description': 'A queen-sized bed for comfortable sleeping.'}, {'name': 'Wardrobe', 'description': 'A tall wardrobe for clothing storage.'}, {'name': 'Desk', 'description': 'A minimalist desk for work or study.'}, {'name': 'Chair', 'description': 'An ergonomic chair for the desk.'}, {'name': 'Nightstand', 'description': 'A small nightstand for bedside essentials.'}]
0it [00:00, ?it/s]You are an seasoned interior designer who is great at creating the